In [1]:
from scripts.densenet_regression import DenseNet
from scripts.slice_selection import *

In [2]:
with tf.device('/gpu:0'):
        model = DenseNet(
            img_dim=(256, 256, 1), 
            nb_layers_per_block=12, nb_dense_block=4, growth_rate=12, nb_initial_filters=16, 
            compression_rate=0.5, sigmoid_output_activation=True, 
            activation_type='relu', initializer='glorot_uniform', output_dimension=1, batch_norm=True )


In [3]:
import warnings
import os 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
warnings.filterwarnings("ignore")   

load_weights = 'L3_Top_Selection_Model_Weight.h5'
model.load_weights('../model/test/' + load_weights)
#print('loaded:' ,model_weight)
print('loaded:' , load_weights)


loaded: L3_Top_Selection_Model_Weight.h5


In [10]:
from scripts.image_processing.image_window import apply_window,get_image_path_by_id
from scipy.ndimage.filters import gaussian_filter
import SimpleITK as sitk
import functools
from skimage.transform import resize
import numpy as np 
import glob


image_dir = '../data/test/input'

#images = sorted(glob.glob(os.path.join(image_dir, '*.nii.gz')))
img_names = os.listdir(image_dir)
#images = [os.path.join(image_dir, x) for x in img_names]
images = [(image_dir + '/'+ x) for x in img_names]

print(len(images),images[0])

2 ../data/test/input/test-volume-11.nii.gz


In [11]:

df_prediction = pd.DataFrame()

for idx, image_path in enumerate(images):
    image_sitk = sitk.ReadImage(image_path)
    patient_id = image_path.split('/')[-1][:-7]
    
    zspacing = image_sitk.GetSpacing()[-1]
    xy_spacing  = image_sitk.GetSpacing()[0]
    image_array  = sitk.GetArrayFromImage(image_sitk)
    windowed_images = apply_window(image_array)

    resize_func = functools.partial(resize, output_shape=model.input_shape[1:3],
                                    preserve_range=True, anti_aliasing=True, mode='constant')
    series = np.dstack([resize_func(im) for im in windowed_images])
    series = np.transpose(series[:, :, :, np.newaxis], [2, 0, 1, 3])
    
    predictions = model.predict(series)
    predictions = 2.0 * (predictions - 0.5)  #sigmoid.output = True
    smoothing_kernel = 2.0
    smoothed_predictions = gaussian_filter(predictions, smoothing_kernel)

    zero_crossings = []
    for s in range(len(smoothed_predictions) - 1):
        if (smoothed_predictions[s] < 0.0) != (smoothed_predictions[s + 1] < 0.0):
            if(abs(smoothed_predictions[s]) < abs(smoothed_predictions[s + 1])):
                zero_crossings.append(s)
            else:
                zero_crossings.append(s + 1)
    if len(zero_crossings) == 0:
        chosen_index = np.argmin(abs(smoothed_predictions))    
    else: 
        chosen_index = sorted(zero_crossings[:1])[0]
            
    df_inter = pd.DataFrame({'patient_id':patient_id,
                                'L3_Predict_slice':chosen_index,
                                'Z_spacing':round(zspacing,5),
                                 'XY_spacing': round(xy_spacing,5)},index=[0])
    df_prediction = df_prediction.append(df_inter)
    df_prediction.to_csv('../data/test/output_csv/L3_Top_Slice_Prediction.csv')
    print(idx,patient_id,'L3_Predict_slice',chosen_index)


7/7 [==============================] - 1s 80ms/step
0 test-volume-11 L3_Predict_slice 125
4/4 [==============================] - 0s 34ms/step
1 test-volume-8 L3_Predict_slice 68


In [12]:
print(df_prediction.shape)
df_prediction.head()

(2, 4)


patient_id  L3_Predict_slice  Z_spacing  XY_spacing
0  test-volume-11               125        2.0     0.71680
0   test-volume-8                68        4.0     0.76953